<a href="https://colab.research.google.com/github/rpasquini/defidata/blob/main/AAVE_Unhealthy_positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unhealthy positions Monitor

In [1]:
import requests
from pprint import pprint
from datetime import datetime
import pandas as pd

In [3]:
#(See https://thegraph.com/hosted-service/subgraph/aave/protocol-v2?version=current)
subgraph_http_address='https://api.thegraph.com/subgraphs/name/aave/protocol-v2'

# function to use requests.post to make an API call to the subgraph url
def run_query(query):

    # endpoint where you are making the request
    request = requests.post(subgraph_http_address,
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

In [4]:
#users(first:1000, skip: """+str(1000*count)+""", orderBy: id, orderDirection: desc, where: {borrowedReservesCount_gt: 0}) {
count=0
query="""{
users(first:10, skip: """+str(1000*count)+""", orderBy: id, orderDirection: desc, where: {borrowedReservesCount_gt: 0}){
          id
          borrowedReservesCount
          collateralReserve:reserves(where: {currentATokenBalance_gt: 0}) {
            currentATokenBalance
            reserve{
              usageAsCollateralEnabled
              reserveLiquidationThreshold
              reserveLiquidationBonus
              borrowingEnabled
              utilizationRate
              symbol
              underlyingAsset
              price {
                priceInEth
              }
              decimals
            }
          }
          borrowReserve: reserves(where: {currentTotalDebt_gt: 0}) {
            currentTotalDebt
            reserve{
              usageAsCollateralEnabled
              reserveLiquidationThreshold
              borrowingEnabled
              utilizationRate
              symbol
              underlyingAsset
              price {
                priceInEth
              }
              decimals
            }
          }
      }
      }
"""
result = run_query(query)
result

{'data': {'users': [{'id': '0xffff8941130157a0153fb5be2618b257f28d3b55',
    'borrowedReservesCount': 1,
    'collateralReserve': [{'currentATokenBalance': '385716688415711350',
      'reserve': {'usageAsCollateralEnabled': True,
       'reserveLiquidationThreshold': '8600',
       'reserveLiquidationBonus': '10500',
       'borrowingEnabled': True,
       'utilizationRate': '0.6195913',
       'symbol': 'WETH',
       'underlyingAsset': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
       'price': {'priceInEth': '1000000000000000000'},
       'decimals': 18}}],
    'borrowReserve': [{'currentTotalDebt': '353684533943004137474',
      'reserve': {'usageAsCollateralEnabled': True,
       'reserveLiquidationThreshold': '9000',
       'borrowingEnabled': True,
       'utilizationRate': '0.29835445',
       'symbol': 'DAI',
       'underlyingAsset': '0x6b175474e89094c44da98b954eedeac495271d0f',
       'price': {'priceInEth': '823421689926389'},
       'decimals': 18}}]},
   {'id': '0xfffcb

In [61]:
result['data']['users'][0]

'0xffff8941130157a0153fb5be2618b257f28d3b55'

In [59]:
def test():
  datauser0=result['data']['users'][0]
  #print('price in ETH', int(datauser0["borrowReserve"][0]['reserve']["price"]["priceInEth"]))
  #print('principal_borrowed', int(datauser0["borrowReserve"][0]["currentTotalDebt"]))
  priceInEth=int(datauser0["borrowReserve"][0]['reserve']["price"]["priceInEth"])*(10**-18)
  principal_borrowed=int(datauser0["borrowReserve"][0]["currentTotalDebt"])*(10**-18)
  
  return priceInEth 

test()


0.38571668841571133

In [62]:
def compute_total_borrowed(userdata):

  """Computes total borrowed  -across all reserves/currencies -(in ETH)
  """
  total_borrowed = 0
  # Three additional variables to detect the largest borrow across all currencies.
  #max_borrowed_symbol = None
  #max_borrowed_principal = 0
  #max_borrowed_price_in_eth = 0

  for borrow_reserve in userdata["borrowReserve"]:
            decimals=int(borrow_reserve["reserve"]["decimals"])
            price_in_eth = int(borrow_reserve["reserve"]["price"]["priceInEth"]) / (10 **decimals )
            principal_borrowed = int(borrow_reserve["currentTotalDebt"]) / (10 **decimals )
            total_borrowed += price_in_eth * principal_borrowed
            
            #if principal_borrowed > max_borrowed_principal:
                #max_borrowed_symbol = borrow_reserve["reserve"]["symbol"]
                #max_borrowed_principal = principal_borrowed
                #max_borrowed_price_in_eth = price_in_eth
  
  return  total_borrowed

def compute_total_collateral_times_thresholds(userdata):

    """Computes total collateral times threshold -across all reserves/currencies - (in ETH)
    """
    total_collateral=0
    total_collateral_threshold=0

    for collateral_reserve in userdata["collateralReserve"]:
        decimals=int(collateral_reserve["reserve"]["decimals"])
        price_in_eth = int(collateral_reserve["reserve"]["price"]["priceInEth"])/ (10 ** decimals)
        principal_a_token_balance = int(collateral_reserve["currentATokenBalance"])/ (10 ** decimals)
        total_collateral +=price_in_eth * principal_a_token_balance
        liquidation_threshold=(int(collateral_reserve['reserve']['reserveLiquidationThreshold']) / 10000)

        #sum of collateral times thresholds 
        total_collateral_threshold += price_in_eth * principal_a_token_balance 
            

    return total_collateral_threshold

def compute_health_factor(userdata):

    """Computes health factor
    """
    total_borrowed=compute_total_borrowed(userdata)
    total_collateral_threshold=compute_total_collateral_times_thresholds(userdata)
    health_factor = total_collateral_threshold / total_borrowed
    
    return health_factor


def parse_users(queryresult):
  health_factors=[]
  for userdata in queryresult['data']['users']:
    health_factor=compute_health_factor(userdata)
    health_factors.append(health_factor)
  return health_factors


  #for borrow_reserve in user["borrowReserve"]:
    #borrow_reserve["reserve"]["price"]["priceInEth"]

In [64]:
parse_users(result)

[1.3244331961924112,
 1.6968409223663327e-12,
 6.349675016212684e-13,
 4.6212726631695325e-12,
 1.2023253276327498e-12,
 2.534265536975705e-12,
 1.0930230251206927e-12,
 0.26135753466403827,
 3.2509595324524424e-12,
 1.2280057397117004]

In [ ]:
compute_total_borrowed(datauser0)
compute_total_collateral_times_thresholds(datauser0)

In [13]:
def parse_users(payload):
    loans = []

    for user in payload["users"]:
        total_borrowed = 0
        total_collateral = 0
        total_collateral_threshold = 0
        max_borrowed_symbol = None
        max_borrowed_principal = 0
        max_borrowed_price_in_eth = 0
        max_collateral_symbol = None
        max_collateral_bonus = 0
        max_collateral_price_in_eth = 0

        for borrow_reserve in user["borrowReserve"]:
            price_in_eth = borrow_reserve["reserve"]["price"]["priceInEth"]
            principal_borrowed = borrow_reserve["currentTotalDebt"]
            total_borrowed += (
                price_in_eth * principal_borrowed
                / 10 ** borrow_reserve["reserve"]["decimals"]
            )
            if principal_borrowed > max_borrowed_principal:
                max_borrowed_symbol = borrow_reserve["reserve"]["symbol"]
                max_borrowed_principal = principal_borrowed
                max_borrowed_price_in_eth = price_in_eth

        for collateral_reserve in user["collateralReserve"]:
            price_in_eth = collateral_reserve["reserve"]["price"]["priceInEth"]
            principal_a_token_balance = collateral_reserve["currentATokenBalance"]
            total_collateral += (
                price_in_eth * principal_a_token_balance
                / 10 ** collateral_reserve["reserve"]["decimals"]
            )
            total_collateral_threshold += (price_in_eth * principal_a_token_balance * (collateral_reserve['reserve']['reserveLiquidationThreshold'] / 10000)) / 10**collateral_reserve['reserve']['decimals']
            if collateral_reserve['reserve']['reserveLiquidationBonus'] > max_collateral_bonus:
              max_collateral_symbol = collateral_reserve['reserve']['symbol']
              max_collateral_bonus = collateral_reserve['reserve']['reserveLiquidationBonus']
              max_collateral_price_in_eth = price_in_eth

    health_factor = total_collateral_threshold / total_borrowed

    if health_factor <= health_factor_max:
        loans.append({
            "user_id": user['id'],
            "healthFactor": health_factor,
            "max_collateralSymbol": max_collateral_symbol,
            "max_borrowedSymbol": max_borrowed_symbol,
            "max_borrowedPrincipal": max_borrowed_principal,
            "max_borrowedPriceInEth": max_borrowed_price_in_eth,
            "max_collateralBonus": max_collateral_bonus / 10000,
            "max_collateralPriceInEth": max

{'errors': [{'locations': [{'line': 2, 'column': 1}],
   'message': 'Unexpected `users[Name]`\nExpected `{`, `query`, `mutation`, `subscription` or `fragment`'}]}

In [ ]:
    for i, borrow_reserve in enumerate(user['borrowReserve']):
        price_in_eth = borrow_reserve['reserve']['price']['priceInEth']
        principal_borrowed = borrow_reserve['currentTotalDebt']
        total_borrowed += (price_in_eth * principal_borrowed) / 10**borrow_reserve['reserve']['decimals']
        if principal_borrowed > max_borrowed_principal:
            max_borrowed_symbol = borrow_reserve['reserve']['symbol']
            max_borrowed_principal = principal_borrowed
            max_borrowed_price_in_eth = price_in_eth

    for i, collateral_reserve in enumerate(user['collateralReserve']):
        price_in_eth = collateral_reserve['reserve']['price']['priceInEth']
        principal_a_token_balance = collateral_reserve['currentATokenBalance']
        total_collateral += (price_in_eth * principal_a_token_balance) / 10**collateral_reserve['reserve']['decimals']
        total_collateral_threshold += (price_in_eth * principal_a_token_balance * (collateral_reserve['reserve']['reserveLiquidationThreshold'] / 10000)) / 10**collateral_reserve['reserve']['decimals']
        if collateral_reserve['reserve']['reserveLiquidationBonus'] > max_collateral_bonus:
            max_collateral_symbol = collateral_reserve['reserve']['symbol']
            max_collateral_bonus = collateral_reserve['reserve']['reserveLiquidationBonus']
            max_collateral_price_in_eth = price_in_eth

    health_factor = total_collateral_threshold / total_borrowed

    if health_factor <= health_factor_max:
        loans.append({
            "user_id": user['id'],
            "healthFactor": health_factor,
            "max_collateralSymbol": max_collateral_symbol,
            "max_borrowedSymbol": max_borrowed_symbol,
            "max_borrowedPrincipal": max_borrowed_principal,
            "max_borrowedPriceInEth": max_borrowed_price_in_eth,
            "max_collateralBonus": max_collateral_bonus / 10000,
            "max_collateralPriceInEth": max
